# House Prices: Advanced Regression Techniques

Kaggle Link: https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview

This problem is all about predicting the price of a house based on various features available. 

# Feature Selection

**We have converted all the categorical, year varaibles into proper numerical format in the Feature engineering notebook. We shall be using that new data format for our next steps of Feature Selection. Feature Selection is really important part in ML model building process because along with using the right features it is equally important to discard the unimportant ones and transform or create new features from existing combination of features. In this notebook I will be using some methods for Feature Selection.**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

pd.pandas.set_option('display.max_columns', None) # to display all columns

In [3]:
data = pd.read_csv('data/X_train.csv')
data.head()

,Id,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_nan,MasVnrArea_nan,GarageYrBlt_nan
0,1,12.247695,0.235294,0.75,0.417704,0.366337,1.0,1.0,0.000000,0.333333,1.0,0.00,0.0,0.636364,0.4,1.0,0.75,1.0,0.666667,0.500,0.949275,0.883333,0.0,0.0,1.0,1.0,0.50,0.783150,0.666667,1.0,1.00,0.75,0.75,0.25,1.000000,0.810016,0.833333,0.0,0.727112,0.821664,1.0,1.00,1.0,1.0,0.356123,0.911138,0.0,0.577685,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.500000,1.0,0.000000,0.2,0.8,0.936364,0.666667,0.50,0.900560,0.666667,1.0,1.0,0.000000,0.745316,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.090909,0.50,0.666667,0.75,0.0,0.0,0.0
1,2,12.109011,0.000000,0.75,0.494579,0.391310,1.0,1.0,0.000000,0.333333,1.0,0.50,0.0,0.500000,0.2,1.0,0.75,0.6,0.555556,0.875,0.753623,0.433333,0.0,0.0,0.4,0.3,0.25,0.000000,0.333333,1.0,0.50,0.75,0.75,1.00,0.666667,0.839799,0.833333,0.0,0.790541,0.856885,1.0,1.00,1.0,1.0,0.503026,0.000000,0.0,0.470213,0.000000,0.5,0.666667,0.0,0.375,0.333333,0.333333,0.333333,1.0,0.333333,0.6,0.8,0.690909,0.666667,0.50,0.882252,0.666667,1.0,1.0,0.883380,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.363636,0.25,0.666667,0.75,0.0,0.0,0.0
2,3,12.317167,0.235294,0.75,0.434407,0.422352,1.0,1.0,0.333333,0.333333,1.0,0.00,0.0,0.636364,0.4,1.0,0.75,1.0,0.666667,0.500,0.934783,0.866667,0.0,0.0,1.0,1.0,0.50,0.763480,0.666667,1.0,1.00,0.75,0.75,0.50,1.000000,0.775892,0.833333,0.0,0.832688,0.828206,1.0,1.00,1.0,1.0,0.383409,0.912543,0.0,0.593068,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.333333,1.0,0.333333,0.6,0.8,0.918182,0.666667,0.50,0.911426,0.666667,1.0,1.0,0.000000,0.702043,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.727273,0.50,0.666667,0.75,0.0,0.0,0.0
3,4,11.849398,0.294118,0.75,0.388076,0.390288,1.0,1.0,0.333333,0.333333,1.0,0.25,0.0,0.727273,0.4,1.0,0.75,1.0,0.666667,0.500,0.311594,0.333333,0.0,0.0,0.2,0.4,0.25,0.000000,0.333333,1.0,0.25,0.50,1.00,0.25,0.666667,0.701797,0.833333,0.0,0.854408,0.810393,1.0,0.75,1.0,1.0,0.399909,0.898872,0.0,0.579130,0.333333,0.0,0.333333,0.0,0.375,0.333333,0.666667,0.416667,1.0,0.333333,0.8,0.4,0.890909,0.333333,0.75,0.917117,0.666667,1.0,1.0,0.000000,0.680915,0.917881,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.090909,0.00,0.666667,0.00,0.0,0.0,0.0
4,5,12.429217,0.235294,0.75,0.512646,0.468754,1.0,1.0,0.333333,0.333333,1.0,0.50,0.0,1.000000,0.4,1.0,0.75,1.0,0.777778,0.500,0.927536,0.833333,0.0,0.0,1.0,1.0,0.50,0.843023,0.666667,1.0,1.00,0.75,0.75,0.75,1.000000,0.803164,0.833333,0.0,0.844750,0.848057,1.0,1.00,1.0,1.0,0.466206,0.932219,0.0,0.666499,0.333333,0.0,0.666667,0.5,0.500,0.333333,0.666667,0.583333,1.0,0.333333,0.6,0.8,0.909091,0.666667,0.75,0.944734,0.666667,1.0,1.0,0.834859,0.782435,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.000000,0.50,0.666667,0.75,0.0,0.0,0.0


In [5]:
# We shall use the conventional names Xtrain and ytrain and divide the data

y_train = data[['SalePrice']] #Target variable/dependent feature
X_train = data.drop(['Id','SalePrice'],axis=1)  # 'Id' column is irrelevant for our next steps

**SelectFromModel is a sklearn function that selects the best features according the model that we use for Feature Selection. We can use models like Linear Regression,Logistic regression etc., to extract features from. Here as part of our work we use Lasso Regression, a variant of Linear Regression for extracting features.**

In [11]:
# A proper 'alpha' needs to be set as parameter. This is the key parameter in selecting the No. of features.
# Higher the 'alpha' value lesser the no. of features selected.

selectedFsModel = SelectFromModel(Lasso(alpha=0.005, random_state=0))
selectedFsModel.fit(X_train, y_train)

SelectFromModel(estimator=Lasso(alpha=0.005, random_state=0))

**We can use Cross Validation for selecting best alpha value. But here I have just used a plain method.**

In [12]:
selectedFsModel.get_support()

array([ True, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False,  True, False,
       False,  True,  True, False, False, False, False, False, False,
       False, False,  True, False,  True, False,  True, False, False,
       False, False, False,  True,  True, False,  True, False, False,
        True, False, False, False, False, False, False,  True, False,
       False,  True,  True,  True, False,  True,  True, False, False,
       False, False,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False])

In [13]:
# lets display the number of total and selected features

selectedFs = X_train.columns[(selectedFsModel.get_support())]

# let's print some stats
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selectedFs)))
print('features with coefficients shrank to zero: {}'.format(
    np.sum(selectedFsModel.estimator_.coef_ == 0)))

total features: 82
selected features: 21
features with coefficients shrank to zero: 61


In [15]:
# features selected
selectedFs

Index(['MSSubClass', 'Neighborhood', 'OverallQual', 'YearRemodAdd',
       'RoofStyle', 'BsmtQual', 'BsmtExposure', 'BsmtFinSF1', 'HeatingQC',
       'CentralAir', '1stFlrSF', 'GrLivArea', 'KitchenQual', 'Fireplaces',
       'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageCars', 'WoodDeckSF',
       'OpenPorchSF', 'SaleCondition'],
      dtype='object')

In [16]:
X_train = X_train[selected_feat] #Filtering only required columns
X_train

,MSSubClass,Neighborhood,OverallQual,YearRemodAdd,RoofStyle,BsmtQual,BsmtExposure,BsmtFinSF1,HeatingQC,CentralAir,1stFlrSF,GrLivArea,KitchenQual,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,WoodDeckSF,OpenPorchSF,SaleCondition
0,0.235294,0.636364,0.666667,0.883333,0.0,0.75,0.25,0.810016,1.00,1.0,0.356123,0.577685,0.666667,0.000000,0.2,0.8,0.666667,0.50,0.000000,0.745316,0.75
1,0.000000,0.500000,0.555556,0.433333,0.0,0.75,1.00,0.839799,1.00,1.0,0.503026,0.470213,0.333333,0.333333,0.6,0.8,0.666667,0.50,0.883380,0.000000,0.75
2,0.235294,0.636364,0.666667,0.866667,0.0,0.75,0.50,0.775892,1.00,1.0,0.383409,0.593068,0.666667,0.333333,0.6,0.8,0.666667,0.50,0.000000,0.702043,0.75
3,0.294118,0.727273,0.666667,0.333333,0.0,0.50,0.25,0.701797,0.75,1.0,0.399909,0.579130,0.666667,0.333333,0.8,0.4,0.333333,0.75,0.000000,0.680915,0.00
4,0.235294,1.000000,0.777778,0.833333,0.0,0.75,0.75,0.803164,1.00,1.0,0.466206,0.666499,0.666667,0.333333,0.6,0.8,0.666667,0.75,0.834859,0.782435,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.235294,0.590909,0.555556,0.833333,0.0,0.75,0.25,0.000000,1.00,1.0,0.396746,0.564405,0.333333,0.333333,0.6,0.8,0.666667,0.50,0.000000,0.696388,0.75
1456,0.000000,0.545455,0.555556,0.633333,0.0,0.75,0.25,0.820289,0.50,1.0,0.690850,0.645786,0.333333,0.666667,0.6,0.8,0.333333,0.50,0.900819,0.000000,0.75
1457,0.294118,0.727273,0.666667,0.933333,0.0,0.50,0.25,0.723860,1.00,1.0,0.480158,0.688647,0.666667,0.666667,0.8,0.8,0.666667,0.25,0.000000,0.743399,0.75
1458,0.000000,0.363636,0.444444,0.766667,1.0,0.50,0.50,0.566353,0.75,1.0,0.443387,0.414465,0.666667,0.000000,0.2,0.8,0.333333,0.25,0.906069,0.000000,0.75


**The above new X-train data can be used for next steps of ML model building.**